In [3]:
from transformers import  Pipeline, set_seed
from datasets import  load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from datasets import load_dataset, load as load_metric
from transformers import  AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
import torch
nltk.download('punkt')

/Users/sanjaymahto/anaconda3/envs/textsum38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjaymahto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import torch

if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")      # Apple-GPU (M1/M2/M3)
elif torch.cuda.is_available():
    device = torch.device("cuda")     # NVIDIA GPU on other machines
else:
    device = torch.device("cpu")      # fallback

print("Running on:", device)


Running on: mps


In [5]:
# device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = 'google/pegasus-cnn_dailymail'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_peasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
# pegasus = Pipeline(
#     "summarization",
#     model=model_peasus,
#     tokenizer=tokenizer,
#     device=0 if torch.cuda.is_available() else -1,
#     max_length=512,
#     min_length=30,
#     length_penalty=2.0,
#     num_beams=4,
#     early_stopping=True
# )
# def summarize_text(text):
#     """
#     Summarizes the input text using the Pegasus model.
#     """
#     # Tokenize the text into sentences
#     sentences = sent_tokenize(text)
    
#     # Join the sentences back into a single string
#     text = ' '.join(sentences)
    
#     # Generate the summary
#     summary = pegasus(text, max_length=130, min_length=30, do_sample=False)
    
#     return summary[0]['summary_text']

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
# ❶ Install wget once if it isn't present
!brew install wget          # or use the curl/Python alternatives above

# ❷ Download
!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip

# ❸ Unzip (optional)
# !unzip -o summarizer-data.zip -d data


zsh:1: command not found: brew
zsh:1: command not found: wget


In [8]:
dataset_samsum = load_from_disk("samsum_dataset")
dataset_samsum
# /Users/sanjaymahto/NLP-1/Text Summarizer/research/data/samsum_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [14]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum.keys()]

print(f"Split lengths: {split_lengths}")
print(f"Features : {dataset_samsum['train'].column_names}")
print("\nDialogues")

print(dataset_samsum['test'][10]['dialogue'])
print("\nSummary")

print(dataset_samsum['test'][10]['summary'])

Split lengths: [14732, 819, 818]
Features : ['id', 'dialogue', 'summary']

Dialogues
Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask

Summary
Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. 


In [15]:
def convert_example_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True, padding='max_length')
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True, padding='max_length')
        
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [16]:
dataset_samsum_pt = dataset_samsum.map(convert_example_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/sanjaymahto/anaconda3/envs/textsum38/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:01<00:00, 812.38 examples/s]


In [17]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [18]:
# Training the model 
from transformers import  DataCollatorForSeq2Seq


seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_peasus)

In [19]:
from transformers import  TrainingArguments, Trainer 

trainer_args = TrainingArguments(
    output_dir="pegasus-samsum",
    num_train_epochs=1, 
    warmup_steps=500,
    per_device_train_batch_size=1, 
    per_device_eval_batch_size=1, 
    weight_decay=0.01,
    logging_steps=10, 
    evaluation_strategy="epoch",
    eval_steps=500,
    save_steps=1e6, 
    gradient_accumulation_steps=16
)

/Users/sanjaymahto/anaconda3/envs/textsum38/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model_peasus, args = trainer_args, 
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsum_pt['test'],
    eval_datase t=dataset_samsum_pt['validation']
)

/var/folders/wg/8rqwg0nd7995t2tzgq40dsph0000gn/T/ipykernel_36369/274080829.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

  0%|          | 0/51 [00:00<?, ?it/s]

In [2]:
from tqdm import tqdm

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """Split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""

    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size=16, device=device,
                                column_text="article", column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))


    for article_batch, target in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)

  ):
        inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                               attention_mask=input['attention_mask'].to(device),
                               length_penalty=0.8,
                               num_beams=8,
                               max_length=128)

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

        metric.add_batch(predictions=decoded_summaries, references=target)
    
    # Finally compute and return the ROUGE Scores, 
    score = metric.compute()
    return score







In [ ]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric("rouge")

In [ ]:
score = calculate_metric_on_test_ds(
    dataset_samsum_pt['test'][1:10],
    rouge_metric, 
    trainer.model, 
    tokenizer,
    batch_size=2, 
    device=device,
    column_text="dialogue",
    column_summary="summary" 
    
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = [f'pegasus'])

In [ ]:
## Save model 
model_peagus.save_pretrained("pegasus-samsum.model")



In [ ]:
## Save tokenizer 
tokenizer.save_pretrained('tokenizer')

In [ ]:
## Load 
tokenizer = AutoTokenizer.from_pretrained('tokenizer')

In [ ]:
# Prediction 
gen_kwargs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128,
}

sample_text = dataset_samsum['test'][0]['dialogue']

reference = dataset_samsum['test'][0]['summary']

pipe = Pipeline(
    "summarization",
    model="model_peagus",
    tokenizer=tokenizer,
    # device=0 if torch.cuda.is_available() else -1,
    # max_length=128,
    # min_length=30,
    # length_penalty=2.0,
    # num_beams=4,
    # early_stopping=True
)

## 
print("Dialogue")
print(sample_text)

print("\nReference Summary: ")
print(reference)

print("\nModel Summary: ")
print(pipe(sample_text, **gen_kwargs)[0]['summary_text'])

In [ ]:
0eb9ebeaf3e16e04e89c4068999b1193eb54255b

In [14]:
# solution('abc', 'bc') # returns true
solution('abc', 'd') # returns false

False

In [13]:
solution('abc', 'bc')

True

In [10]:
start = 'abc'
end = 'bc'


In [11]:
n = len(end)
print(n)
start = start[-n:]
print(start)

2
bc


In [12]:
def solution(starting, ending):
    n = len(ending)
    starting = starting[-n:]
    if starting == ending: 
        return True 
    else:
        return False
    